In [1]:
from src.search_engine import SearchEngine
from src.model import Model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [3]:
def get_best_using_sparse(query, candidates, vectorizer):
    tfidf_vectors = vectorizer.transform(candidates).toarray()
    tfidf_query = vectorizer.transform([query]).toarray()
    best_dist = -99999999999
    best_idx = -1 
    for i in range(tfidf_vectors.shape[0]):
        # print(tfidf_query.shape, tfidf_vectors.shape)
        sim = cosine_sim(np.ravel(tfidf_query), np.ravel(tfidf_vectors[i]))
        if sim > best_dist:
            best_dist = sim
            best_idx = i
    return best_idx

In [4]:
preprocessed_data_dir = "data/preprocessed_corpus.json"
embeddings_dir = "data/corpus_embeddings.json"

In [5]:
model = Model("distiluse-base-multilingual-cased-v1")
search = SearchEngine(embeddings_dir, preprocessed_data_dir, model)

In [6]:
known_definitions = [text.split(".")[0] for text in search.text]
len(known_definitions)

100848

In [7]:

vectorizer = TfidfVectorizer()
tfidf_vectors = vectorizer.fit_transform(search.text)

In [8]:
all_questions = []
with open("data/task2_questions_with_answers.tsv", encoding="utf-8") as f:
    for line in f.readlines():
        line = line.strip().split("\t")
        question, answer = line[0], line[1:]
        # question = " ".join(question.split(" ")[3:])
        all_questions.append((question, answer))
all_questions = all_questions[:10]
print(all_questions)

[('Jak nazywa się pierwsza litera alfabetu greckiego?', ['alfa']), ('Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?', ['cięciwa']), ('W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?', ['w Egipcie']), ('Czy w państwach starożytnych powoływani byli posłowie i poselstwa?', ['tak']), ('W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”?', ['Mazowsze']), ('W którym państwie leży Bombaj?', ['w Indiach']), ('Który numer boczny nosi czołg Rudy z „Czterech pancernych”?', ['102']), ('Co budował w Egipcie inżynier Tarkowski, ojciec Stasia?', ['Kanał Sueski']), ('Czy owoce niektórych kaktusów są jadalne?', ['tak']), ('Kwartet – to ilu wykonawców?', ['czterech', 'czworo', '4'])]


In [9]:
dense_correct = 0
dense_sparse_correct = 0
for query, answers in all_questions:
    print("Query: {}, Answers: {}".format(query, answers))
    dense_result = search.run(query)[0]
    result_word = dense_result.split(".")[0]
    dense_correct += int(result_word in answers)
    print("Best text using dense rep: \n\t{}".format(dense_result), end="")
    candidates = search.run(query, k_best=20)
    idx_result = get_best_using_sparse(query, candidates, vectorizer)
    print(idx_result)
    result = candidates[idx_result]
    word_result = result.split(".")[0]
    dense_sparse_correct += int(word_result in answers)
    print("Best text using dense and sparse rep:\n\t{}".format(result))

Query: Jak nazywa się pierwsza litera alfabetu greckiego?, Answers: ['alfa']
Best text using dense rep: 
	alfa. Definicja:  {{jęz}} nazwa pierwszej litery alfabetu greckiego, α; {{wikipedia}}
0
Best text using dense and sparse rep:
	alfa. Definicja:  {{jęz}} nazwa pierwszej litery alfabetu greckiego, α; {{wikipedia}}

Query: Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?, Answers: ['cięciwa']
Best text using dense rep: 
	apsyda. Definicja:  {{astr}} jeden z dwóch skrajnych punktów orbity eliptycznej;
5
Best text using dense and sparse rep:
	cięciwa. Definicja:  {{geom}} odcinek łączący dwa dowolne punkty krzywej lub powierzchni, który nie ma innych punktów wspólnych z daną krzywą czy powierzchnią; {{wikipedia}}

Query: W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?, Answers: ['w Egipcie']
Best text using dense rep: 
	szałaput. Definicja:  {{książk}} {{żart}} ktoś pusty i lekkomyślny
4
Best text using dense and sparse rep:
	powieściowy. Definicja:  z

In [10]:
print("DENSE:\n Correct answers: {} ({:.2f}%)".format(dense_correct, dense_correct * 100/ len(all_questions)))
print("DENSE + SPARSE:\n Correct answers: {} ({:.2f}%)".format(dense_sparse_correct, dense_sparse_correct * 100/ len(all_questions)))

DENSE:
 Correct answers: 1 (10.00%)
DENSE + SPARSE:
 Correct answers: 2 (20.00%)
